## Reducing portfolio risk by diversification
## compiler: Mashele Given Phazamisa

In [1]:
### Import the necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from datetime import  datetime,timedelta
from scipy.optimize import minimize  ## scientific python function to deal with optimisation

## Part 1:
1. Create a vector of weights which sum up to 1
2. Calculate the annualised Expected return for the portfolio
3. Calculate the annualised standard deviation for the portfolio.
4. Calculate the annualised Expected Return and Standard Deviation for the market portfolio

### Ten Stock dataset analysis

In [2]:
## Now lets read the 10 stock prices dataset
data = pd.read_csv('stock_prices_data.csv',sep=';')

In [3]:
## lets display the five rows of the dataset
data.head()

,Date,SPY,BND,GLD,NFLX,VTI,IBM,AAPL,TSLA,VZ,GE
0,2019/01/30 00:00,267.579987,79.720001,124.690002,340.660004,137.320007,128.470367,41.312500,20.584667,54.000000,54.644810
1,2019/01/31 00:00,269.929993,80.089996,124.750000,339.500000,138.529999,128.508606,41.610001,20.468000,55.060001,61.010029
2,2019/02/01 00:00,270.059998,79.669998,124.500000,339.850006,138.729996,128.202683,41.630001,20.813999,54.549999,61.190174
3,2019/02/04 00:00,271.959992,79.599998,123.959999,351.339996,139.720001,129.244736,42.812500,20.859333,54.040001,61.310272
4,2019/02/05 00:00,273.100006,79.769997,124.279999,355.809998,140.300003,129.588913,43.544998,21.423332,54.139999,63.832344


In [4]:
### Lets set the index as Date
data.set_index('Date',inplace=True)

In [5]:
data.head()

,SPY,BND,GLD,NFLX,VTI,IBM,AAPL,TSLA,VZ,GE
Date,,,,,,,,,,
2019/01/30 00:00,267.579987,79.720001,124.690002,340.660004,137.320007,128.470367,41.312500,20.584667,54.000000,54.644810
2019/01/31 00:00,269.929993,80.089996,124.750000,339.500000,138.529999,128.508606,41.610001,20.468000,55.060001,61.010029
2019/02/01 00:00,270.059998,79.669998,124.500000,339.850006,138.729996,128.202683,41.630001,20.813999,54.549999,61.190174
2019/02/04 00:00,271.959992,79.599998,123.959999,351.339996,139.720001,129.244736,42.812500,20.859333,54.040001,61.310272
2019/02/05 00:00,273.100006,79.769997,124.279999,355.809998,140.300003,129.588913,43.544998,21.423332,54.139999,63.832344


### Stock returns calculations

In [43]:
### Create a function to calcualate the portfolio risk
def getPort_Expected_Return(init_weights):
    # Now lets calculate the return on each stock
    returns_df = data.pct_change(1).dropna()
    # let create a code to find the number of stocks
    num_stocks = len(returns_df.columns)
    # Lets calculate the mean returns on each stock
    expected_mean_return= returns_df.mean()
    ### Lets compute the portfolio daily expected return
    port_daily_expected_return = np.dot(np.transpose(init_weights),expected_mean_return)
    ### Now lets compute the annualised portfolio return:using sophisticated method
    port_annual_expected_return = (((1+port_daily_expected_return)**250 -1 ))
    return port_annual_expected_return

In [44]:
# Create the weights for each dataframe: assume equal weightings
num_stocks = len(data.columns)
weights = [1/num_stocks]*num_stocks

In [45]:
## Lets now compute the annualised expected return of the portfolio:
print(f'The annualised portfolio expected return is:{getPort_Expected_Return(init_weights=weights)}')

The annualised portfolio expected return is:0.20968977159275792


### Compute the annualised standard deviation of the portfolio

In [9]:
### Create a function to calcualate the portfolio risk
def getPortRisk(init_weights):
    # Now lets calculate the return on each stock
    returns_df = data.pct_change(1).dropna()
    # let create a code to find the number of stocks
    num_stocks = len(returns_df.columns)
    # Lets calcuate the variance covariance matrix
    vcv = returns_df.cov() 
    # Now lets compute the variance of each portfolio created
    var_p = np.dot(np.transpose(init_weights),np.dot(vcv,init_weights))
    # now lets compute the daily portfolio standard deviation
    sd_p = np.sqrt(var_p)
    # now lets compute the annualised standard deviation for the portfolio
    annual_sd_p =  sd_p*np.sqrt(250)
    return annual_sd_p

In [10]:
# calculate the weights for each dataframe: assume equal weightings
num_stocks = len(data.columns)
init_weights = [1/num_stocks]*num_stocks

In [11]:
## Lets now compute the annualised expected return of the portfolio:
print(f'The annualised portfolio standard deviation is:{getPortRisk(init_weights)}')

The annualised portfolio standard deviation is:0.19248567484945814


### Market Portfolio Analysis

In [12]:
# Lets read the market portfolio dataset
df = pd.read_csv('sp500_price.csv')

In [13]:
df.head()

,Date,sp500
0,01/03/2012 16:00,1277.06
1,01/04/2012 16:00,1277.30
2,01/05/2012 16:00,1281.06
3,01/06/2012 16:00,1277.81
4,01/09/2012 16:00,1280.70


In [14]:
### Lets the index as Date
df.set_index('Date',inplace=True)

In [15]:
df.head()

,sp500
Date,
01/03/2012 16:00,1277.06
01/04/2012 16:00,1277.30
01/05/2012 16:00,1281.06
01/06/2012 16:00,1277.81
01/09/2012 16:00,1280.70


In [16]:
## Code to compute the returns on each timestamp and drop the missing values
market_returns = df.pct_change(1).dropna()

In [17]:
market_returns.rename(columns={'sp500':'returns'},inplace=True)

In [18]:
market_returns.head()

,returns
Date,
01/04/2012 16:00,0.000188
01/05/2012 16:00,0.002944
01/06/2012 16:00,-0.002537
01/09/2012 16:00,0.002262
01/10/2012 16:00,0.008886


In [19]:
### Lets compute the daily market returns:
daily_market_return = market_returns['returns'].mean()

In [20]:
daily_market_return

0.0005191262038891577

In [21]:
### Now lets compute the annualised market portfolio using the sophisticated method
annual_market_expected_return = (1+daily_market_return)**250-1

In [46]:
annual_market_expected_return

0.13854129369128

In [23]:
## Compute the daily market standard deviation.
daily_sd_market= np.sqrt(np.var(market_returns['returns'],ddof=1))

In [24]:
daily_sd_market

0.007576893043007455

In [47]:
annualised_market_sd= daily_sd_market*np.sqrt(250)
print(annualised_market_sd)

0.11980119801693842


## Part 2
1. Optimise the portfolio weights so that the expected return is equal to the annualised return on the market portfolio.i.e 
                                          i.e E(R)-Target return=0
2. Optimise the portfolio weights so that the risk of the chosen stocks portfolio is equal to the annualised risk of the market portfolio.
3. Optimise the portfolio weights so that the risk of the portfolio is minimised.
4. Insights and recommendations

## Optimise the portfolio weights so that the expected return is equal to the annualised return on the market portfolio.i.e 
   Objective function:  E(R)= Target return(Market portfolio)
   Constraints:  
   1. The sum of the weights must be equal to 1. np.sum(weights)-1 = 0
   2. The difference between the expected return and the market return = 0 . i.e E(R)-Target return=0
   Bounds:
   1. The weights should between 0 and 1.


In [26]:
target_return = annual_market_expected_return   # set the target return
target_risk = annualised_market_sd  # set the target risk

In [27]:
target_return

0.13854129369128

In [28]:
### Create a function to calcualate the portfolio risk
def getPortReturn(init_weights):
    # Now lets calculate the return on each stock
    returns_df = data.pct_change(1).dropna()
    ### Lets compute the portfolio annual expected return using sophisticated method
    port_annual_expected_return = (1+ np.dot(np.transpose(init_weights),returns_df.mean()))**250-1
    return port_annual_expected_return

In [29]:
 returns_df = data.pct_change(1).dropna()

In [30]:
## We need create a bound that the weights should be between 0 and 1
bounds = tuple((0,1) for i in range(num_stocks))

In [31]:
## Lets now include other constraints that weights show sum to 1
cons = ({'type':'eq','fun':lambda x:np.sum(x)-1},
        {'type':'eq','fun':lambda x: ((1+ np.dot(np.transpose(x),returns_df.mean()))**250-1) -target_return })

In [32]:
# calculate the weights for each dataframe: assume equal weightings
num_stocks = len(data.columns)
init_weights = [1/num_stocks]*num_stocks

In [33]:
getPortReturn(init_weights)

0.20968977159275792

In [34]:
### Lets call the minimize method:
results = minimize(fun=getPortReturn,x0=init_weights,bounds=bounds,constraints=cons)

In [35]:
results

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 0.1385412936919117
       x: [ 1.086e-01  1.360e-01  1.172e-01  9.634e-02  1.095e-01
            1.140e-01  7.106e-02  2.120e-02  1.378e-01  8.831e-02]
     nit: 4
     jac: [ 1.608e-01 -1.825e-02  1.044e-01  2.406e-01  1.546e-01
            1.252e-01  4.056e-01  7.309e-01 -2.988e-02  2.930e-01]
    nfev: 44
    njev: 4

In [36]:
# Lets display the weights to minimize the portfolio return
results['x']

array([0.10856363, 0.13600517, 0.11720411, 0.09634054, 0.10951077,
       0.11401647, 0.07105962, 0.02120388, 0.13778705, 0.08830875])

In [37]:
# Lets confirm the target expected return
getPortReturn(results['x'])

0.1385412936919117

In [38]:
## Lets now calculate the new portfolio risk using the optimal weight
getPortRisk(results['x'])

0.16092290186068836

In [39]:
target_risk

0.11980119801693842

In [48]:
# Lets create a data frame for weights that optimise the portfolio return
optimised_weights = pd.DataFrame(results['x'])

In [49]:
optimised_weights

,0
0,0.108564
1,0.136005
2,0.117204
3,0.096341
4,0.109511
5,0.114016
6,0.071060
7,0.021204
8,0.137787
9,0.088309


In [50]:
##Lets change thier index
optimised_weights.index= data.columns
optimised_weights.rename(columns={optimised_weights.columns[0]:'weights'},inplace=True)

In [51]:
optimised_weights

,weights
SPY,0.108564
BND,0.136005
GLD,0.117204
NFLX,0.096341
VTI,0.109511
IBM,0.114016
AAPL,0.071060
TSLA,0.021204
VZ,0.137787
GE,0.088309


In [52]:
### For readability lets change the rounding
optimised_weights['weights_rounded'] = round(optimised_weights['weights'],3)

In [53]:
optimised_weights

,weights,weights_rounded
SPY,0.108564,0.109
BND,0.136005,0.136
GLD,0.117204,0.117
NFLX,0.096341,0.096
VTI,0.109511,0.110
IBM,0.114016,0.114
AAPL,0.071060,0.071
TSLA,0.021204,0.021
VZ,0.137787,0.138
GE,0.088309,0.088


In [54]:
## Lets sort the rounded values in ascending order
optimised_weights['weights_rounded'].sort_values(ascending=False)

VZ      0.138
BND     0.136
GLD     0.117
IBM     0.114
VTI     0.110
SPY     0.109
NFLX    0.096
GE      0.088
AAPL    0.071
TSLA    0.021
Name: weights_rounded, dtype: float64

In [55]:
## Lets sort the rounded values in ascending order
optimised_weights['weights_rounded'].sort_values(ascending=False).cumsum()

VZ      0.138
BND     0.274
GLD     0.391
IBM     0.505
VTI     0.615
SPY     0.724
NFLX    0.820
GE      0.908
AAPL    0.979
TSLA    1.000
Name: weights_rounded, dtype: float64

## 2. Optimise the portfolio weights so that the risk of the chosen stocks portfolio is equal to the annualised risk of the market portfolio.
 Objective function:  Stock expected risk = Target risk(Market portfolio)
 Constraints:  
   1. The sum of the weights must be equal to 1. np.sum(weights)-1 = 0
   2. The difference between the expected risk portfolio and the market portfolio risk = 0 
   Bounds:
   1. The weights should between 0 and 1.


In [57]:
# Lets set the target risk
target_risk = annualised_market_sd 

In [58]:
target_risk 

0.11980119801693842

In [59]:
### Create a function to calcualate the portfolio risk
def getPortRisk2(init_weights):
    # Now lets calculate the return on each stock
    returns_df = data.pct_change(1).dropna()
    # now lets compute the daily portfolio standard deviation
    annual_sd_p = (np.sqrt(np.dot(np.transpose(init_weights),np.dot(returns_df.cov(),init_weights))))*np.sqrt(250)
    
    return annual_sd_p

In [60]:
getPortRisk2(init_weights)

0.19248567484945814

In [61]:
### Lets set the constraints for the risk portfolio
## Lets now include other constraints that weights show sum to 1
cons1 = ({'type':'eq','fun':lambda x:np.sum(x)-1},
         {'type':'eq','fun':lambda x: (np.sqrt(np.dot(np.transpose(x),np.dot(returns_df.cov(),x))))
          *np.sqrt(250)- target_risk })

In [62]:
### Lets call the minimize method:
results_risk = minimize(fun=getPortRisk2,x0=init_weights,bounds=bounds,constraints=cons1)

In [63]:
results_risk

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 0.11980119805974626
       x: [ 9.118e-02  2.255e-01  2.121e-01  2.519e-02  8.743e-02
            1.101e-01  4.574e-02  0.000e+00  1.624e-01  4.028e-02]
     nit: 5
     jac: [ 1.832e-01  2.476e-02  6.108e-02  2.023e-01  1.864e-01
            1.928e-01  2.212e-01  2.423e-01  1.363e-01  2.402e-01]
    nfev: 55
    njev: 5

In [64]:
results_risk['x']

array([0.09117761, 0.22554452, 0.21213337, 0.02519191, 0.08743266,
       0.11012546, 0.04573819, 0.        , 0.16237699, 0.04027929])

In [65]:
## Lets now compute the new expected returns:
getPortReturn(results_risk['x'])

0.08378569498971755

In [66]:
# Lets create a data frame for weights that optimise the portfolio return
optimised_weights1 = pd.DataFrame(results_risk['x'])

In [67]:
optimised_weights1

,0
0,0.091178
1,0.225545
2,0.212133
3,0.025192
4,0.087433
5,0.110125
6,0.045738
7,0.000000
8,0.162377
9,0.040279


In [68]:
##Lets change thier index
optimised_weights1.index= data.columns
optimised_weights1.rename(columns={optimised_weights1.columns[0]:'weights'},inplace=True)

In [71]:
optimised_weights1.round(3)

,weights
SPY,0.091
BND,0.226
GLD,0.212
NFLX,0.025
VTI,0.087
IBM,0.110
AAPL,0.046
TSLA,0.000
VZ,0.162
GE,0.040


In [90]:
optimised_weights1['weights'].sort_values(ascending=False).cumsum()

BND     0.225545
GLD     0.437678
VZ      0.600055
IBM     0.710180
SPY     0.801358
VTI     0.888791
AAPL    0.934529
GE      0.974808
NFLX    1.000000
TSLA    1.000000
Name: weights, dtype: float64

In [73]:
returns_df.head()

,SPY,BND,GLD,NFLX,VTI,IBM,AAPL,TSLA,VZ,GE
Date,,,,,,,,,,
2019/01/31 00:00,0.008782,0.004641,0.000481,-0.003405,0.008811,0.000298,0.007201,-0.005668,0.019630,0.116484
2019/02/01 00:00,0.000482,-0.005244,-0.002004,0.001031,0.001444,-0.002381,0.000481,0.016904,-0.009263,0.002953
2019/02/04 00:00,0.007035,-0.000879,-0.004337,0.033809,0.007136,0.008128,0.028405,0.002178,-0.009349,0.001963
2019/02/05 00:00,0.004192,0.002136,0.002581,0.012723,0.004151,0.002663,0.017109,0.027038,0.001850,0.041136
2019/02/06 00:00,-0.001318,-0.000376,-0.006759,-0.010174,-0.000998,0.005680,0.000345,-0.012852,-0.006465,-0.015052


In [76]:
# Lets compute the individual daily standard deviation:
sd_annual = (returns_df.std())*np.sqrt(250)

In [77]:
sd_annual

SPY     0.208948
BND     0.067841
GLD     0.148854
NFLX    0.458298
VTI     0.213886
IBM     0.263447
AAPL    0.316323
TSLA    0.643349
VZ      0.209404
GE      0.407912
dtype: float64

In [78]:
sd_annual_df = pd.DataFrame(sd_annual)

In [79]:
sd_annual_df.head()

,0
SPY,0.208948
BND,0.067841
GLD,0.148854
NFLX,0.458298
VTI,0.213886


In [80]:
sd_annual_df.rename(columns={sd_annual_df.columns[0]:'individual risk'},inplace=True)

In [81]:
sd_annual_df

,individual risk
SPY,0.208948
BND,0.067841
GLD,0.148854
NFLX,0.458298
VTI,0.213886
IBM,0.263447
AAPL,0.316323
TSLA,0.643349
VZ,0.209404
GE,0.407912


In [82]:
sd_annual_df['individual risk'].sort_values(ascending=False)

TSLA    0.643349
NFLX    0.458298
GE      0.407912
AAPL    0.316323
IBM     0.263447
VTI     0.213886
VZ      0.209404
SPY     0.208948
GLD     0.148854
BND     0.067841
Name: individual risk, dtype: float64

### Observation:
* TESLA,NFLX,GE,APPL,IBM constitute the highest risk as compared to others.
* BND is the risky stock.

In [88]:
optimised_weights1.round(3)

,weights
SPY,0.091
BND,0.226
GLD,0.212
NFLX,0.025
VTI,0.087
IBM,0.110
AAPL,0.046
TSLA,0.000
VZ,0.162
GE,0.040


In [84]:
### Lets compute the correlation matrix of the stocks
corr_matrix = returns_df.corr()

In [85]:
corr_matrix.round(2)

,SPY,BND,GLD,NFLX,VTI,IBM,AAPL,TSLA,VZ,GE
SPY,1.00,0.18,0.11,0.48,0.99,0.65,0.81,0.49,0.45,0.60
BND,0.18,1.00,0.37,0.13,0.19,0.07,0.15,0.08,0.11,0.02
GLD,0.11,0.37,1.00,0.10,0.11,0.08,0.09,0.08,0.09,-0.02
NFLX,0.48,0.13,0.10,1.00,0.48,0.13,0.47,0.36,0.14,0.18
VTI,0.99,0.19,0.11,0.48,1.00,0.64,0.79,0.51,0.43,0.61
IBM,0.65,0.07,0.08,0.13,0.64,1.00,0.44,0.19,0.43,0.50
AAPL,0.81,0.15,0.09,0.47,0.79,0.44,1.00,0.49,0.29,0.38
TSLA,0.49,0.08,0.08,0.36,0.51,0.19,0.49,1.00,0.06,0.24
VZ,0.45,0.11,0.09,0.14,0.43,0.43,0.29,0.06,1.00,0.28
GE,0.60,0.02,-0.02,0.18,0.61,0.50,0.38,0.24,0.28,1.00
